In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

from selenium.webdriver import ChromeOptions

option = ChromeOptions()

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
option.add_argument(UA.get_User_Agent_Selenium())
option.add_argument('blink-settings=imagesEnabled=false')
# option.add_argument('--headless')
option.add_extension(Proxy.get_Proxy_Selenium('coroutine'))

option.page_load_strategy = 'eager'

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for b in range(10):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()
                browser.get(input_.loc[a, 'Url'])

                time.sleep(1)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)
                browser.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.1);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.2);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.3);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.4);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.5);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.6);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.7);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.8);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.9);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)

                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//h1[@class="part-detail-title"]')))
                title = browser.find_element(by=By.XPATH, value='//h1[@class="part-detail-title"]').text.strip()

                # = = = = = = = = = = = = = = =

                wait.until(EC.presence_of_element_located((By.XPATH, '//p[@class="part-detail-title-number"]/strong')))
                part_number = browser.find_element(by=By.XPATH, value='//p[@class="part-detail-title-number"]/strong').text.strip()

                # = = = = = = = = = = = = = = =

                wait.until(EC.presence_of_element_located((By.XPATH, '//p[@class="item-description"]')))
                description = browser.find_element(by=By.XPATH, value='//p[@class="item-description"]').text.split('\n')[0].strip()

                # = = = = = = = = = = = = = = =

                wait.until(EC.presence_of_element_located((By.XPATH, '//img[@id="primaryImage"]')))
                src = browser.find_element(by=By.XPATH, value='//img[@id="primaryImage"]').get_attribute('src').strip()

                # = = = = = = = = = = = = = = =

                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="attribute-container flex-container flex-wrap"]/div')))
                list_div = browser.find_elements(by=By.XPATH, value='//div[@class="attribute-container flex-container flex-wrap"]/div')

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': part_number,
                                         'Title': title,
                                         'Description': description,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for i in range(int(len(list_div) / 2)):
                    df_temp.loc[0, list_div[i*2].text.strip()[:-1]] = list_div[i*2+1].text.strip()

                browser.quit()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                break
            except:
                browser.quit()
                time.sleep(0.3)
                continue

                # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'序号': int(input_.loc[a, '序号']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part_-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：746

[ok] - https://www.summitracing.com/parts/aaf-all26240
[尝试次数：1] - [剩余数量：739] - [当前时间：20:20:34]

[ok] - https://www.summitracing.com/parts/ado-10-9246
[尝试次数：1] - [剩余数量：738] - [当前时间：20:20:35]

[ok] - https://www.summitracing.com/parts/sum-122241
[尝试次数：1] - [剩余数量：737] - [当前时间：20:20:35]

[ok] - https://www.summitracing.com/parts/ado-10-9289
[尝试次数：1] - [剩余数量：736] - [当前时间：20:20:37]

[ok] - https://www.summitracing.com/parts/ado-11562588
[尝试次数：1] - [剩余数量：735] - [当前时间：20:20:49]

[ok] - https://www.summitracing.com/parts/ado-10-9290
[尝试次数：1] - [剩余数量：734] - [当前时间：20:20:50]

[ok] - https://www.summitracing.com/parts/ado-11589253
[尝试次数：2] - [剩余数量：733] - [当前时间：20:20:56]

[ok] - https://www.summitracing.com/parts/ado-11570082
[尝试次数：4] - [剩余数量：732] - [当前时间：20:21:03]

[ok] - https://www.summitracing.com/parts/ado-12612350
[尝试次数：1] - [剩余数量：731] - [当前时间：20:21:09]

[ok] - https://www.summitracing.com/parts/ado-24203590
[尝试次数：3] - [剩余数量：730] - [当前时间：20:21:18]

[ok] - https://www.summitracing.com/

Traceback (most recent call last):
  File "/Users/hzm/Desktop/YangTeng/code/venv/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3652, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: '序号'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "src/gevent/greenlet.py", line 908, in gevent._gevent_cgreenlet.Greenlet.run
  File "/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_5901/1189026643.py", line 166, in crawler
    df_temp = pd.DataFrame([{'序号': int

[ok] - https://www.summitracing.com/parts/chp-lsnx3050
[尝试次数：2] - [剩余数量：690] - [当前时间：20:24:44]

[ok] - https://www.summitracing.com/parts/chp-lsnx3055
[尝试次数：2] - [剩余数量：689] - [当前时间：20:24:50]

[ok] - https://www.summitracing.com/parts/cmb-08-0127
[尝试次数：1] - [剩余数量：688] - [当前时间：20:24:58]

[ok] - https://www.summitracing.com/parts/cmb-08-0129
[尝试次数：1] - [剩余数量：687] - [当前时间：20:25:12]

[ok] - https://www.summitracing.com/parts/cmb-08-0130
[尝试次数：1] - [剩余数量：686] - [当前时间：20:25:33]

[ok] - https://www.summitracing.com/parts/cmb-08-0059
[尝试次数：1] - [剩余数量：685] - [当前时间：20:26:33]

[ok] - https://www.summitracing.com/parts/chp-ls3158l3
[尝试次数：2] - [剩余数量：684] - [当前时间：20:26:35]

[ok] - https://www.summitracing.com/parts/cmb-08-0132
[尝试次数：1] - [剩余数量：683] - [当前时间：20:26:54]

[ok] - https://www.summitracing.com/parts/chp-ls1150-l3
[尝试次数：5] - [剩余数量：682] - [当前时间：20:26:56]

[ok] - https://www.summitracing.com/parts/cmb-08-0131
[尝试次数：1] - [剩余数量：681] - [当前时间：20:27:08]

[ok] - https://www.summitracing.com/parts/cmb

Traceback (most recent call last):
  File "/Users/hzm/Desktop/YangTeng/code/venv/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3652, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: '序号'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "src/gevent/greenlet.py", line 908, in gevent._gevent_cgreenlet.Greenlet.run
  File "/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_5901/1189026643.py", line 166, in crawler
    df_temp = pd.DataFrame([{'序号': int

[ok] - https://www.summitracing.com/parts/cmb-08-0210
[尝试次数：3] - [剩余数量：604] - [当前时间：20:35:41]

[ok] - https://www.summitracing.com/parts/cmb-08-0211
[尝试次数：2] - [剩余数量：603] - [当前时间：20:35:49]

[ok] - https://www.summitracing.com/parts/cmb-08-0209
[尝试次数：2] - [剩余数量：602] - [当前时间：20:35:49]

[ok] - https://www.summitracing.com/parts/cmb-08-0212
[尝试次数：1] - [剩余数量：601] - [当前时间：20:35:54]

[ok] - https://www.summitracing.com/parts/cmb-08-0207
[尝试次数：5] - [剩余数量：600] - [当前时间：20:35:56]

[ok] - https://www.summitracing.com/parts/ctr-12-200
[尝试次数：1] - [剩余数量：599] - [当前时间：20:35:59]

[ok] - https://www.summitracing.com/parts/ctr-13-270
[尝试次数：1] - [剩余数量：598] - [当前时间：20:36:03]

[ok] - https://www.summitracing.com/parts/ctr-13-274blk
[尝试次数：2] - [剩余数量：597] - [当前时间：20:36:13]

[ok] - https://www.summitracing.com/parts/ctr-13-280a
[尝试次数：3] - [剩余数量：596] - [当前时间：20:36:17]

[ok] - https://www.summitracing.com/parts/ctr-12-280a
[尝试次数：2] - [剩余数量：595] - [当前时间：20:36:18]

[ok] - https://www.summitracing.com/parts/ctr-15-2

KeyboardInterrupt
2024-03-25T12:40:45Z


KeyboardInterrupt: 